[View in Colaboratory](https://colab.research.google.com/github/venkatakula9652/Complaints_Framework/blob/master/Complaints_Framework_Model_Venkatesh.ipynb)

In [0]:
# IMPORTING NECESSARY PACKAGES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
data = pd.read_csv("drive/Complaints_Dataset/complaints-2018-07-20_02_20.csv")

In [8]:
data.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,02/05/13,Mortgage,FHA mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,"BAYVIEW LOAN SERVICING, LLC",MA,02151,NaN,NaN,Web,02/06/13,Closed with explanation,Yes,No,301664
1,05/15/17,Mortgage,Other type of mortgage,Trouble during payment process,NaN,NaN,Company has responded to the consumer and the ...,BMO HARRIS BANK NATIONAL ASSOCIATION,IL,60165,NaN,NaN,Fax,05/17/17,Closed with explanation,Yes,NaN,2484441
2,09/24/14,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Ditech Financial LLC,FL,34209,NaN,NaN,Referral,10/01/14,Closed with explanation,Yes,Yes,1043621
3,11/06/15,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,OCWEN LOAN SERVICING LLC,GA,30253,NaN,NaN,Referral,11/09/15,Closed with explanation,Yes,No,1643449
4,03/08/12,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,"CITIBANK, N.A.",NJ,07438,NaN,NaN,Referral,03/08/12,Untimely response,No,No,32218


In [0]:
data = data[data.Company == 'WELLS FARGO & COMPANY']

In [0]:
data.head()

In [0]:
data.shape

In [0]:
#req_cols = ['Date received','Issue','State','Submitted via', 'Date sent to company','Company reponse to consumer']
req_cols = ['Date received','Issue','State','Submitted via', 'Date sent to company','Company response to consumer']

In [0]:
data.columns

In [0]:
data_req = data[req_cols]

In [0]:
data_req.shape

In [0]:
data_req.columns

In [0]:
data_req['Date received']=pd.to_datetime(data_req['Date received'])
data_req['Date sent to company']=pd.to_datetime(data_req['Date sent to company'])

In [0]:
data_req.info()

In [0]:
data_req['Total Days'] = data_req['Date sent to company']-data_req['Date received']

In [0]:
data_req.dtypes

In [0]:
data_req=data_req.drop(data_req.columns[[0, 4]], axis=1)

In [0]:
data_req.info()

In [0]:
data_req['Issue'].value_counts(dropna=False)

In [0]:
data_req['State'].value_counts(dropna=False)

In [0]:
data_req['Submitted via'].value_counts(dropna=False)

In [0]:
data_req['Company response to consumer'].value_counts(dropna=False)

In [0]:
data_req['Total Days'].value_counts(dropna=False)

In [0]:
data_req=data_req.dropna(how='any')  

In [0]:
data_req.info()

In [0]:
data_req[:5]['Total Days'] / np.timedelta64(1, 's')

In [0]:
data_req.info()

In [0]:
dumy=pd.get_dummies(data_req['Issue'])
dumy.head()

In [0]:
data_req_new = pd.concat([data_req,dumy],axis=1)
data_req_new.shape

In [0]:
dumy=pd.get_dummies(data_req['State'])
dumy.head()

In [0]:
data_req_new = pd.concat([data_req_new,dumy],axis=1)
data_req_new.shape

In [0]:
dumy=pd.get_dummies(data_req['Submitted via'])
dumy.head()

In [0]:
data_req_new = pd.concat([data_req_new,dumy],axis=1)
data_req_new.shape

In [0]:
dumy=pd.get_dummies(data_req['Company response to consumer'])
dumy.head()

In [0]:
data_req_new = pd.concat([data_req_new,dumy],axis=1)
data_req_new.shape

In [0]:
data_req_new=data_req_new.drop(['Issue','State','Submitted via','Company response to consumer'], axis=1)
data_req_new.shape

In [0]:
!pip install kmodes

In [0]:
from kmodes.kmodes import KModes


In [0]:
km = KModes(n_clusters=10, init='Huang', n_init=5, verbose=1)

In [0]:
clusters = km.fit_predict(data_req_new)

In [0]:
print(km.cluster_centroids_)

In [0]:
clusters=clusters.reshape(34113,1)

In [0]:
class_cluster = pd.DataFrame(clusters,columns=['Class cluster'])

In [0]:
class_cluster.head()

In [0]:
data_req_new.head()

In [0]:
data_req_new.index = range(data_req_new.shape[0])
data_req_new.head()

In [0]:
data_req_new = pd.concat([data_req_new,class_cluster],axis=1)

In [0]:
data_req_new.head()